#### Install Necessary Libraries

In [1]:
!pip install sqlalchemy beautifulsoup4 selenium requests


   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   --- ------------------------------------ 0.8/9.5 MB 5.6 MB/s eta 0:00:02
   ---------- ----------------------------- 2.6/9.5 MB 8.4 MB/s eta 0:00:01
   ----------------- ---------------------- 4.2/9.5 MB 8.4 MB/s eta 0:00:01
   --------------------- ------------------ 5.2/9.5 MB 7.2 MB/s eta 0:00:01
   ------------------------- -------------- 6.0/9.5 MB 7.2 MB/s eta 0:00:01
   -------------------------------- ------- 7.9/9.5 MB 6.8 MB/s eta 0:00:01
   ----------------------------------- ---- 8.4/9.5 MB 6.8 MB/s eta 0:00:01
   ---------------------------------------- 9.5/9.5 MB 6.2 MB/s eta 0:00:00
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0


#### Import Libraries

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv


### Test for response 200

In [31]:
url = 'https://www.aliexpress.com/w/wholesale-laptop.html?page=1&g=y&SearchText=laptop'

r = requests.get(url)

print(r)

<Response [200]>


In [32]:
# Configure Selenium ChromeDriver options

options = Options()
options.headless = True
service = Service(executable_path = r'New folder\chromedriver-win64\chromedriver.exe')

# Initialize the webdriver
driver = webdriver.Chrome(service = service, options = options)

# Define the URL
url = 'https://www.aliexpress.com/w/wholesale-laptop.html?page=1&g=y&SearchText=laptop'

# Use Selenium to open the page


# Wait for the dynamic content to load
time.sleep(10)

# Get the page source and close the browser
page_source = driver.page_source
driver.quit()



### Extraction

In [ ]:
# Configure Selenium ChromeDriver options

options = Options()
options.headless = True
service = Service(executable_path = r'New folder\chromedriver-win64\chromedriver.exe')

# Initialize the webdriver
driver = webdriver.Chrome(service = service, options = options)

# Lists to store extracted data
product_names = []
prices = []
store_names = []
store_links = []
shipping_prices = []
discounts = []
item_sold = []
original_prices = []
shipping_free_statuses = []


# Assuming the total number of pages is known 
total_pages = 60

for page_number in range(1, total_pages + 1):
    url = f"https://www.aliexpress.com/w/wholesale-laptop.html?page={page_number}&g=y&SearchText=laptop"
    driver.get(url)
    time.sleep(10)

    # Use BeautifulSoup to parse the loaded page_source
    
    #page_source = driver.page_source
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    #adjust the class selectors based on the current web site structure 
    laptops = soup.find_all("div", class_ = "hz_bu search-item-card-wrapper-gallery")

    for laptop in laptops:
        #product_names
        try:
            product_name = laptop.find("div", class_ = "lq_ae").text
        except AttributeError:
            product_name = ""
    

        #prices
        price = laptop.find("div", class_ = "lq_j3").text.replace("C$", "").replace(",", "")

        #store_name
        store_name = laptop.find("a", class_ = "io_ir" ).text

        #store_link
        try:
            store_link = laptop.find("a", class_ = "io_ir" )["href"]
        except (TypeError, KeyError):
            store_link = ""

        #shipping price
        try:
            shipping_price = laptop.find("span", class_ = "mi_az mi_l6").text
        except AttributeError:
            shipping_price = ""

        #extra discount
        try:
            discount = laptop.find("span", class_ = "lq_eu").text
        except AttributeError:
            discount = ""
        
        #item sold
        try:
            item_sold_text = laptop.find("span", class_ = "lq_jg").text
            item_sold_value = item_sold_text.split()[0] if item_sold_text else ""
        except AttributeError:
            item_sold_value = ""
        

        # Append data to list

        product_names.append(product_name)
        prices.append(price)
        store_names.append(store_name)
        store_links.append(store_link)
        shipping_prices.append(shipping_price)
        discounts.append(discount)
        item_sold.append(item_sold_value)
       
        # Extracting additional information
        former_price = laptop.find_all("div", class_ = "lq_j4")
        original_prices.append(former_price[0].text if former_price else "")

        free_shipping = laptop.find_all("div", class_ = "lq_lv")
        ship_free = free_shipping[0].text.strip() if free_shipping else ""
        shipping_free_statuses.append(ship_free)

#Once all the pages information extracted, quit the driver
driver.quit()

# Create a Dictionary of the lists, then create a DataFrame from the Dictionary 
data = {
    "Product_Name" : product_names,
    "Price" : prices,
    "Store_Name" : store_names,
    "Store_Links" : store_links,
    "Shipping_Price" : shipping_price,
    "Discount" : discounts,
    "Item_Sold" : item_sold,
    "Original_Price" : original_prices,
    "Free_Shipping" : shipping_free_statuses
}

df = pd.DataFrame(data)




In [95]:
# Display the dataframe
display(df.head(5))

,Product_Name,Price,Store_Name,Store_Links,Shipping_Price,Discount,Item_Sold,Original_Price,Free_Shipping
0,CHUWI MiniBook X Laptop Tablet 2 In 1 Intel N1...,474.69,CHUWI Choice Store,//www.aliexpress.com/store/1102773765,,,900+,,Free shipping
1,FIREBAT A14 Laptop Intel N5095 14.1 Inch 16GB ...,333.76,Big Save Store,https://www.aliexpress.com/ssr/300001505/rEkz8...,,,307,,Free shipping
2,Firebat A16 Laptop 16 Inch CPU Intel N5095 16G...,350.55,Firebat Choice Store,//www.aliexpress.com/store/1103843268,,-68%,184,"C$1,125.47",Free shipping
3,11.6 inch Portable monitor 1366X768 lcd displa...,57.59,Shop1102854679 Store,//www.aliexpress.com/store/1102854679,,-60%,"5,000+",C$146.11,Free shipping
4,BYONE 11.6 Inch Mini Laptop Intel Celeron N400...,196.52,BYONE Official Store,//www.aliexpress.com/store/900254088,,-41%,129,C$337.89,


In [35]:
df.shape

(720, 9)

In [36]:
# Save the raw 
df.to_csv('Aliexpress_laptops.csv', index = False)

## Data Transformation

In [37]:
df.columns

Index(['Product_Name', 'Price', 'Store_Name', 'Store_Links', 'Shipping_Price',
       'Discount', 'Item_Sold', 'Original_Price', 'Free_Shipping'],
      dtype='object')

The dataframe above can be split into different tables before loading into the database, the tables are split based on related attributes

In [40]:
# products table

products_columns = ['Product_Name', 'Price', 'Store_Name', 'Store_Links']
products = df[products_columns].copy().drop_duplicates().reset_index(drop = True)

# create product id
products.index.name = 'product_id'
products = products.reset_index()


In [96]:
products.head()

,product_id,Product_Name,Price,Store_Name,Store_Links
0,0,CHUWI MiniBook X Laptop Tablet 2 In 1 Intel N1...,474.69,CHUWI Choice Store,//www.aliexpress.com/store/1102773765
1,1,FIREBAT A14 Laptop Intel N5095 14.1 Inch 16GB ...,333.76,Big Save Store,https://www.aliexpress.com/ssr/300001505/rEkz8...
2,2,Firebat A16 Laptop 16 Inch CPU Intel N5095 16G...,350.55,Firebat Choice Store,//www.aliexpress.com/store/1103843268
3,3,11.6 inch Portable monitor 1366X768 lcd displa...,57.59,Shop1102854679 Store,//www.aliexpress.com/store/1102854679
4,4,BYONE 11.6 Inch Mini Laptop Intel Celeron N400...,196.52,BYONE Official Store,//www.aliexpress.com/store/900254088


In [ ]:
# Discount table

ddiscount_columns = ['Product_Name', 'Price', 'Original_Price', 'Discount']
discounts = df[discount_columns].copy()

discounts.index.name = 'discount_id'
discounts = discounts.reset_index()

# Function to clean price-related columns
def clean_currency(column):
    return (
        column.astype(str)  # Ensure column is string
        .str.replace(r'[^\d.]', '', regex=True)  # Remove everything except numbers and dots
        .replace('', '0')  # Replace empty strings with '0' to avoid conversion errors
        .astype(float)  # Convert to float
    )

# Clean 'Original_Price' and 'Price'
discounts['Original_Price'] = clean_currency(discounts['Original_Price'])
discounts['Price'] = clean_currency(discounts['Price'])

# Clean 'Discount' column: Remove '%' and '-' then rename to 'Discount%'
discounts['Discount'] = (
    discounts['Discount']
    .astype(str)  # Ensure it's a string
    .str.replace(r'[%\-]', '', regex=True)  # Remove '%' and '-'
    .replace('', '0')  # Replace empty values with '0'
    .astype(float)  # Convert to float
)
discounts = discounts.rename(columns={"Discount": "Discount%"})

# Fill missing values with 0.0
discounts.fillna({
    'Original_Price': 0.0,
    'Discount%': 0.0,
    'Price': 0.0
}, inplace=True)

print(discounts.dtypes)  # Check data types
discounts.head()  # Preview cleaned data


discount_id         int64
Product_Name       object
Price             float64
Original_Price    float64
Discount%         float64
dtype: object


,discount_id,Product_Name,Price,Original_Price,Discount%
0,0,CHUWI MiniBook X Laptop Tablet 2 In 1 Intel N1...,474.69,0.00,0.0
1,1,FIREBAT A14 Laptop Intel N5095 14.1 Inch 16GB ...,333.76,0.00,0.0
2,2,Firebat A16 Laptop 16 Inch CPU Intel N5095 16G...,350.55,1125.47,68.0
3,3,11.6 inch Portable monitor 1366X768 lcd displa...,57.59,146.11,60.0
4,4,BYONE 11.6 Inch Mini Laptop Intel Celeron N400...,196.52,337.89,41.0


In [86]:
# sales table 
sales = discounts.merge(df, on = 'Product_Name', how = 'left')

sales = sales[['Product_Name', 'Price_x', 'Item_Sold', 'Store_Name']].drop_duplicates()
sales = sales.rename(columns={"Price_x": "Price"})

sales.index.name = 'Sales_ID'
sales.reset_index()

,Sales_ID,Product_Name,Price,Item_Sold,Store_Name
0,0,CHUWI MiniBook X Laptop Tablet 2 In 1 Intel N1...,474.69,900+,CHUWI Choice Store
1,1,CHUWI MiniBook X Laptop Tablet 2 In 1 Intel N1...,474.69,"1,000+",Stone's Store
2,2,FIREBAT A14 Laptop Intel N5095 14.1 Inch 16GB ...,333.76,307,Big Save Store
3,3,FIREBAT A14 Laptop Intel N5095 14.1 Inch 16GB ...,333.76,307,Stone's Store
4,5,Firebat A16 Laptop 16 Inch CPU Intel N5095 16G...,350.55,184,Firebat Choice Store
...,...,...,...,...,...
628,791,FRESH&CLEAN New Acer Predator Helios 300 Gamin...,476.19,,Electronic Trading International Ltd Store
629,802,"NEW Original Gaming Laptop with 17"" Screen, As...",976.80,,Shop1103688357 Store
630,813,Best trade for new Razer Blade 16 (2024) Gamin...,2075.70,,Shop912720244 Store
631,836,HOT SALES Promo for Notebook Zenbook Duo OLED ...,512.82,,Shop912360237 Store


In [87]:
# shipping table

shipping_columns = ['Product_Name', 'Shipping_Price', 'Free_Shipping']
shipping = df[shipping_columns].copy()

shipping.index.name = 'shipping_id'
shipping = shipping.reset_index()

shipping.fillna({
    'Shipping_Price' : 0.0,
}, inplace = True)

In [88]:
shipping

,shipping_id,Product_Name,Shipping_Price,Free_Shipping
0,0,CHUWI MiniBook X Laptop Tablet 2 In 1 Intel N1...,,Free shipping
1,1,FIREBAT A14 Laptop Intel N5095 14.1 Inch 16GB ...,,Free shipping
2,2,Firebat A16 Laptop 16 Inch CPU Intel N5095 16G...,,Free shipping
3,3,11.6 inch Portable monitor 1366X768 lcd displa...,,Free shipping
4,4,BYONE 11.6 Inch Mini Laptop Intel Celeron N400...,,
...,...,...,...,...
715,715,"NEW Original Gaming Laptop with 17"" Screen, As...",,
716,716,Best trade for new Razer Blade 16 (2024) Gamin...,,
717,717,MSI Katana GF66 15.6 Gaming Laptop i7- 11800H ...,,
718,718,HOT SALES Promo for Notebook Zenbook Duo OLED ...,,


In [89]:
# saving to CSVs

products.to_csv('products.csv', index = False)
discounts.to_csv('discounts.csv', index = False)
sales.to_csv('sales.csv', index = False)
shipping.to_csv('shipping.csv', index = False)


### LOADING TO POSTGRES SQL SERVER

In [3]:
# Retrieve password from .env file

load_dotenv()

password = os.getenv("password")

In [4]:
# The password contains an @ symbol, this section of the code will avoid error due to this.

import urllib.parse

raw_password = password
encoded_password = urllib.parse.quote_plus(raw_password) # this takes care of the @ in the password



In [ ]:
db_params = {
    'username': 'postgres',
    'password': encoded_password,
    'host': 'localhost',
    'port': '5432',
    'database': 'Jumba_Stores'
}

#define database connection url
db_url = f"postgresql://{db_params['username']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}"

# create the database engine with the databse url specified
engine = create_engine(db_url)

# connect to postgreSQL server

with engine.connect() as connection:
    #create the the table and load the data
   
    products.to_sql('products_table', connection, index = False, if_exists = 'replace')
    discounts.to_sql('discounts_table', connection, index = False, if_exists = 'replace')
    sales.to_sql('sales_table', connection, index = False, if_exists = 'replace')
    shipping.to_sql('shipping_table', connection, index = False, if_exists = 'replace')


print('Database, tables and data loaded successfully')


Database, table and data loaded successfully
